In [ ]:
wav_path = "/Users/cirorandazzo/code/callback-analysis/data/loom-only/or14pu27-loom_only-20240813120025-Stim0-Block0.wav"

import numpy as np
import pandas as pd
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib widget

%load_ext autoreload
%autoreload 1

fs, audio = wavfile.read(wav_path)

cam_audio = audio[:,1]
cam_audio.shape

In [ ]:
%aimport utils.video
from utils.video import get_video_frames_from_callback_audio

vid_frames = get_video_frames_from_callback_audio(cam_audio)
vid_frames
print(f'Found {len(vid_frames)} video frames.')

In [ ]:
fig, axs = plt.subplots(nrows=2, sharex=True)

# audio samples between subsequent frames
deltas = vid_frames[1:] - vid_frames[:-1]
deltas = [np.NaN] + list(deltas)

axs[0].scatter(vid_frames, deltas, marker="x")
axs[0].set(
    title="Audio samples between frames",
    ylim=(0, 400),
    ylabel="# audio samples since last frame",
)

# frame tracking in audio channel
t = np.max(cam_audio) * 0.2  # default threshold function

axs[1].plot(  # plot frames
    cam_audio,
    marker="x",
)
axs[1].scatter(  # plot audio samples with detected frame
    vid_frames,
    cam_audio[vid_frames],
    c="r",
    zorder=3,
)
axs[1].plot(  # plot threshold
    [0, len(cam_audio)],
    [t, t],
    c="k",
)
axs[1].set(
    title="Video frame triggers (wav file)",
    xlabel="Audio samples",
)

plt.show()

In [ ]:
import time

xmin_start = 0  # where to start animation
xrange = 0.2  # length of x-axis
dx = 0.025
x_stop = 35

fig = plt.figure()
ax = fig.subplots()

x_time = np.arange(len(cam_audio)) / 44100
ax.plot(x_time, cam_audio)
ax.scatter(vid_frames / 44100, cam_audio[vid_frames], color="red", zorder=3)
ax.set_xlabel("Time since video onset (s)")


def update_xlim(frame):
    global xmin
    global xrange
    global dx
    global ax

    if frame == 0:
        xmin = xmin_start
    else:
        xmin += dx

    if frame % 10 == 0:
        ax.set_title(f"Frame {frame}")

    ax.set_xlim([xmin, xmin + xrange])

    return ax


n_frames = round((x_stop - xmin_start) / dx)
# n_frames = 100

sim = animation.FuncAnimation(
    fig, update_xlim, frames=n_frames, interval=75, repeat=True
)

plt.show()